In [0]:
import pandas as pd
import numpy as np
import pickle
import time
from datetime import date

import matplotlib.pyplot as plt
import seaborn as sns

import os
import gc
import zipfile
import io
from google.colab import files

In [0]:
import tensorflow as tf
import keras
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.model_selection import KFold,train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [0]:
uploaded = files.upload()

Saving train_cleaned.csv.zip to train_cleaned.csv.zip


In [0]:
! unzip train_cleaned.csv.zip

Archive:  train_cleaned.csv.zip
  inflating: train_cleaned.csv       


In [0]:
train = pd.read_csv('train_cleaned.csv')

In [0]:
train.head()

,Unnamed: 0,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,channel,adult_count,child_count,room_count,destination_id,destination_type_id,is_booking,similar_events,hotel_continent,hotel_country,hotel_market,hotel_cluster,stay_duration,plan_time
0,6,2,3,66,189,10067,2014.665587,501,0,0,2,2,0,1,8267,1,0,2,2,50,675,41,1.0,15.0
1,7,2,3,66,189,10067,2014.665587,501,0,1,2,2,0,1,8267,1,0,1,2,50,675,41,1.0,15.0
2,9,2,3,66,189,10067,2014.665587,501,0,0,2,2,0,1,8267,1,0,1,2,50,675,70,1.0,15.0
3,10,2,3,66,189,10067,2014.665587,501,0,0,2,2,0,1,8267,1,0,1,2,50,675,98,1.0,15.0
4,43,30,4,195,991,47725,2014.665587,1048,1,0,9,2,0,1,8803,1,0,1,3,151,1197,5,2.0,215.0


In [0]:
train.drop(columns=['Unnamed: 0'], inplace=True)

#### Train, test, split

In [0]:
train['hotel_cluster'].value_counts()

91    55754
41    41797
48    39949
64    39626
5     32914
65    32783
98    32019
59    30603
70    30271
42    29740
Name: hotel_cluster, dtype: int64

In [0]:
train['hotel_cluster'].replace([91, 41, 48, 64, 5, 65, 98, 59, 70, 42], \
                               ['ninety-one','forty-one','forty-eight','sixty-four', 'five',\
                               'sixty-five','ninety-eight','fifty-nine','seventy','forty-two'], inplace=True)
train['hotel_cluster'].value_counts()

ninety-one      55754
forty-one       41797
forty-eight     39949
sixty-four      39626
five            32914
sixty-five      32783
ninety-eight    32019
fifty-nine      30603
seventy         30271
forty-two       29740
Name: hotel_cluster, dtype: int64

In [0]:
X = train.drop(['hotel_cluster'], axis=1)
y = train['hotel_cluster']

In [0]:
from sklearn.preprocessing import LabelEncoder

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y)
encoded_y = encoder.transform(y)
# convert integers to dummy variables (i.e. one hot encoded)
y = np_utils.to_categorical(encoded_y)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [0]:
# scales data between 0 and 1
X_train = keras.utils.normalize(X_train, axis=1)  
X_test = keras.utils.normalize(X_test, axis=1)

In [0]:
X_train.shape

(292364, 22)

### KERAS
https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/

https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/

In [0]:
def c_model():

  model = Sequential()
  model.add(Dense(2048, activation='relu', input_dim=22))
  # model.add(Dense(1024, activation='relu'))
  # model.add(Dropout(0.25))
  model.add(Dense(512, activation='relu'))
  # model.add(Dropout(0.25))
  model.add(Dense(256, activation='relu'))
  # model.add(Dropout(0.25))
  # model.add(Dense(128, activation='relu'))
  # model.add(Dropout(0.25))
  model.add(Dense(64, activation='relu'))
  # model.add(Dropout(0.5))
  model.add(Dense(10, activation='softmax'))
  
  model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
  
  return model

In [0]:
model = KerasClassifier(build_fn=c_model, epochs=100, batch_size=128)

In [0]:
model.fit(X_train, y_train)

Epoch 1/100
292364/292364 [==============================] - 23s 78us/step - loss: 2.1926 - acc: 0.2014
Epoch 2/100
292364/292364 [==============================] - 21s 71us/step - loss: 2.0742 - acc: 0.2481
Epoch 3/100
292364/292364 [==============================] - 20s 69us/step - loss: 1.9533 - acc: 0.2886
Epoch 4/100
292364/292364 [==============================] - 20s 69us/step - loss: 1.9276 - acc: 0.2977
Epoch 5/100
292364/292364 [==============================] - 20s 69us/step - loss: 1.9110 - acc: 0.3024
Epoch 6/100
292364/292364 [==============================] - 20s 69us/step - loss: 1.8951 - acc: 0.3089
Epoch 7/100
292364/292364 [==============================] - 20s 69us/step - loss: 1.8841 - acc: 0.3119
Epoch 8/100
292364/292364 [==============================] - 20s 69us/step - loss: 1.8691 - acc: 0.3163
Epoch 9/100
292364/292364 [==============================] - 20s 69us/step - loss: 1.8626 - acc: 0.3179
Epoch 10/100
292364/292364 [==============================] - 20

## GridSearchCV
#### Tune batch size and epochs

In [0]:
model = KerasClassifier(build_fn=c_model)

batch_sizes = [128] #batch_size = 128
epochs = [30] #, 50, 100, 200]
parameters = {'epochs': epochs}
clf = GridSearchCV(model, parameters, cv=2)
clf.fit(X_train, y_train)

In [0]:
print(clf.best_score_, clf.best_params_)
means = clf.cv_results_['mean_test_score']
parameters = clf.cv_results_['params']
for mean, parameter in zip(means, parameters):
  print(mean, parameter)

#### Tune optimizer

In [0]:
def c_model(optimizer):

  model = Sequential()
  model.add(Dense(2048, activation='relu', input_dim=22))
  model.add(Dense(512, activation='relu'))
  model.add(Dense(256, activation='relu'))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(10, activation='softmax'))
  
  model.compile(optimizer=optimizer,
                loss='categorical_crossentropy',
                metrics=['accuracy'])
  
  return model

model = KerasClassifier(build_fn=c_model, epochs=10, batch_size=128)
parameters = {'optimizer':['RMSprop','Adagrad','Adam','Adamax']}
clf = GridSearchCV(model, parameters)
clf.fit(X_train, y_train)

In [0]:
print(clf.best_score_, clf.best_params_)
means = clf.cv_results_['mean_test_score']
parameters = clf.cv_results_['params']
for mean, parameter in zip(means, parameters):
  print(mean, parameter)

0.31756303785630646 {'optimizer': 'Adam'}
0.28852731526514486 {'optimizer': 'RMSprop'}
0.2824013900458307 {'optimizer': 'Adagrad'}
0.31756303785630646 {'optimizer': 'Adam'}
0.30522225718563567 {'optimizer': 'Adamax'}


#### Tune activation function

In [0]:
def c_model(activation):

  model = Sequential()
  model.add(Dense(2048, activation=activation, input_dim=22))
  model.add(Dense(512, activation=activation))
  model.add(Dense(256, activation=activation))
  model.add(Dense(64, activation=activation))
  model.add(Dense(10, activation='softmax'))
  
  model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
  
  return model

model = KerasClassifier(build_fn=c_model, epochs=10 , batch_size=128)
parameters = {'activation':['softmax', 'softplus','relu','tanh','sigmoid','hard_sigmoid','linear']}
clf = GridSearchCV(model, parameters)
clf.fit(X_train, y_train)

In [0]:
print(clf.best_score_, clf.best_params_)
means = clf.cv_results_['mean_test_score']
parameters = clf.cv_results_['params']
for mean, parameter in zip(means, parameters):
  print(mean, parameter)

0.32606955712986296 {'activation': 'tanh'}
0.15242300693543726 {'activation': 'softmax'}
0.169446306658109 {'activation': 'softplus'}
0.3091009837076906 {'activation': 'relu'}
0.32606955712986296 {'activation': 'tanh'}
0.15242300693543726 {'activation': 'sigmoid'}
0.15242300693543726 {'activation': 'hard_sigmoid'}
0.2097932714021173 {'activation': 'linear'}


In [0]:
from keras.models import load_model

In [0]:
model.save('model.h5')
files.download('keras_model.h5')

AttributeError: ignored

In [0]:
import sklearn
sorted(sklearn.metrics.SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'brier_score_loss',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'max_error',
 'mutual_info_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'v_measure_score']